pytorch sentencepiece jumanpp

# install

In [ ]:
!apt install libeigen3-dev libprotobuf-dev protobuf-c-compiler

In [ ]:
!pip install pyknp
!pip install datasets transformers rhoknp sentencepiece

In [ ]:
!wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc3/jumanpp-2.0.0-rc3.tar.xz
!tar -xvJof jumanpp-2.0.0-rc3.tar.xz
%cd jumanpp-2.0.0-rc3
!mkdir build
%cd build
!rm -f CMakeCache.txt
!cmake .. -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=/usr/local
!make
!sudo make install

In [ ]:
!echo 'こんにちは，今日は晴れです' | jumanpp

# プログラム

In [1]:
import torch
torch.cuda.is_available()
print(torch.cuda.device_count())
print(torch.cuda.current_device())


1
0


In [12]:
pd.read_json("/home/nb-user/ksl_pub/suggestive_anger/data/fuman_survey/json/human_data_ver5.json")

,fulltext,sentences,danwa_result,result_form,label
0,最近の内容は、のび太が、他力本願ですぐ調子にのり、エゴが強すぎると感じている。子供に見せたく...,"[最近の内容は、のび太が、他力本願ですぐ調子にのり、エゴが強すぎると感じている, 子供に見せ...",原因結果,"{'3': '1.0', '0': '1.0', '1': '7.0', '2': '4.0'}",1
1,ヤクルトの舌や喉に何か残る感じの、あれは何でしょう?すごくいやだ。,[ヤクルトの舌や喉に何か残る感じの、あれは何でしょう?すごくいやだ],詳細化,"{'0': '7.0', '1': '2.0', '3': '2.0', '2': '1.0...",0
2,めばちこ痛い。治ってもまたできる...,"[めばちこ痛い, 治ってもまたできる...]",並列,"{'0': '1.0', '5': '12.0'}",5
3,タケダは、からだ浸透補水液は病院の消毒液を連想させる味がします。体に良くても飲み物としてなじ...,"[タケダは、からだ浸透補水液は病院の消毒液を連想させる味がします, 体に良くても飲み物として...",例提示,"{'2': '1.0', '1': '3.0', '4': '1.0', '3': '8.0'}",3
4,チョコリングの個性が弱くて商品として地味で美味しくない。,[チョコリングの個性が弱くて商品として地味で美味しくない],順接,"{'0': '1.0', '2': '8.0', '5': '1.0', '1': '3.0'}",2
...,...,...,...,...,...
1795,以前みたいに古本の買取販売サービスを行ってほしい。,[以前みたいに古本の買取販売サービスを行ってほしい],短文,"{'2': '4.0', '5': '3.0', '1': '2.0'}",2
1796,前は小さいコップでドリンク飲み放題だったから、3種類ぐらい飲めた。メニューが変わってて、ビー...,"[前は小さいコップでドリンク飲み放題だったから、3種類ぐらい飲めた, メニューが変わってて、...",要望,"{'2': '5.0', '1': '3.0', '3': '1.0'}",2
1797,友達が不満買取センターから、『犯罪予告、また犯罪を助長する投稿』などという理由でログイン停止...,[友達が不満買取センターから、『犯罪予告、また犯罪を助長する投稿』などという理由でログイン停...,質問応答,"{'2': '6.0', '1': '3.0'}",2
1798,面白かったから続編をして欲しい,[面白かったから続編をして欲しい],順接,"{'2': '2.0', '5': '6.0', '1': '1.0'}",5


In [2]:
import pandas as pd

danwasMap = {'並列':0, '対比':1, '要望':2, '条件':3, '原因結果':4, '例提示':5, '詳細化':6, '主題連鎖':7, '焦点主題連鎖':8,'順接':9, '逆接':10, '短文':11, '質問応答':12, '理由':13}
labelsMap = {0: "感情的攻撃", 1: "感情的説得", 2: "理性的説得", 3: "嫌味皮肉", 4: "遠回し", 5: "怒っていない"}
baseData = pd.read_json("/home/nb-user/ksl_pub/suggestive_anger/data/fuman_survey/json/human_data_ver5.json")
baseData = baseData.rename(columns={'fulltext': 'text', "label": "labels"})
baseData = baseData[["text", "labels"]]

In [3]:
def split_test(baseData):
    # 各ラベルごとにデータを分割し、30件をテストデータに使用する
    test_data = pd.DataFrame()
    train_data = pd.DataFrame()

    for label in baseData['labels'].unique():
        label_data = baseData[baseData['labels'] == label]
        label_test_data = label_data.sample(n=30, random_state=42)
        label_train_data = label_data.drop(label_test_data.index)
        
        test_data = pd.concat([test_data, label_test_data])
        train_data = pd.concat([train_data, label_train_data])

    # 確認
    print("Train data:")
    print(train_data['labels'].value_counts())
    print("Test data:")
    print(test_data['labels'].value_counts())
    return train_data, test_data

In [5]:
from sklearn.model_selection import train_test_split
train, test = split_test(baseData)
# train, test = train_test_split(baseData, test_size=0.1, stratify=baseData.labels)
# train.to_csv('/home/nb-user/train.csv')
# test.to_csv('/home/nb-user/test.csv')

Train data:
1    565
2    550
3    153
5    121
0    119
4    112
Name: labels, dtype: int64
Test data:
1    30
0    30
5    30
3    30
2    30
4    30
Name: labels, dtype: int64


In [6]:
from datasets import load_dataset, Features, Value, ClassLabel
import datasets
human_dataset = datasets.DatasetDict({
        "train":datasets.Dataset.from_pandas(train),
        "test":datasets.Dataset.from_pandas(test)
     })

human_dataset


# dataset_files = {
#     "train": ["/content/gdrive/MyDrive/LAB/研究/train.csv"],
#     "test": ["/content/gdrive/MyDrive/LAB/研究/test.csv"],
# }
# human_dataset = load_dataset("csv", data_files=dataset_files)
# human_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 1620
    })
    test: Dataset({
        features: ['text', 'labels', '__index_level_0__'],
        num_rows: 180
    })
})

In [7]:
#human_dataset["test"]["labels"]

In [8]:
from transformers import AutoTokenizer
model_ckpt = "nlp-waseda/roberta-base-japanese-with-auto-jumanpp"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [9]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

def tokenize(batch):
    #tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
    return tokenizer(batch["text"], padding=True, truncation=True)


human_dataset_encoded = human_dataset.map(tokenize, batched=True, batch_size=None)
human_dataset_encoded

Map:   0%|          | 0/1620 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/180 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1620
    })
    test: Dataset({
        features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 180
    })
})

In [ ]:
# human_dataset_encoded
# human_dataset_encoded["train"]["labels"]
# human_dataset_encoded["test"]["labels"]

In [13]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
import torch

num_labels = 6
device = torch.device("cuda", index=0)
model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

batch_size = 32
logging_steps = len(human_dataset_encoded["train"]) // batch_size
output_dir = f"/home/nb-user/output/roberta-waseda"
training_args = TrainingArguments(output_dir=output_dir,
                                  num_train_epochs=10,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False, 
                                  log_level="error",
                                  )

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics=compute_metrics, 
                 train_dataset=human_dataset_encoded["train"], 
                 eval_dataset=human_dataset_encoded["test"],
                 tokenizer=tokenizer)

trainer.train()

/home/nb-user/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.441100,1.668799,0.300000,0.218174
2,1.153300,1.529804,0.366667,0.320074
3,0.933600,1.497454,0.394444,0.356589
4,0.733000,1.585544,0.427778,0.411692
5,0.557500,1.603627,0.466667,0.449842
6,0.442300,1.857624,0.427778,0.409052
7,0.325000,1.838180,0.461111,0.445780
8,0.233500,1.830187,0.466667,0.458343
9,0.204900,1.855875,0.472222,0.465954
10,0.165700,1.873219,0.472222,0.466509


TrainOutput(global_step=510, training_loss=0.6098691837460387, metrics={'train_runtime': 331.6339, 'train_samples_per_second': 48.849, 'train_steps_per_second': 1.538, 'total_flos': 1573481175184800.0, 'train_loss': 0.6098691837460387, 'epoch': 10.0})

In [ ]:
trainer.test()